In [5]:
# Sujet :
# Prédire le pays gagnant lors des JO 2024 en terme de :
# nombre de médailles total ? points de médailles ?
# prédire le total de médailles de chaque pays et faire un classement


# relier host de result pour pouvoir exlure les jeux d'hiver

In [6]:
# Import des biliothèques
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.multioutput import MultiOutputRegressor

In [8]:
# Chargement des jeux de données
from sqlalchemy import create_engine

username = 'jo2024_fatima'
password = 'Ipssi2024!'
hostname = 'postgresql-jo2024.alwaysdata.net'
database = 'jo2024_postgres'

connection_url = f'postgresql://{username}:{password}@{hostname}:5432/{database}'

engine = create_engine(connection_url)

df_result = pd.read_sql("olympic_results", engine)
df_hosts = pd.read_sql("olympic_hosts", engine)

In [9]:
df_result["country_name"].unique()

array(['Republic of Korea', 'Latvia', 'United States of America',
       'Russian Federation', 'Norway', 'Soviet Union', 'Greece',
       'Federal Republic of Germany', 'Sweden', 'Czech Republic',
       'Switzerland', 'German Democratic Republic (Germany)', 'Finland',
       'Czechoslovakia', 'Italy', 'Mexico', 'Argentina', 'Japan',
       'Independent Olympic Athletes', 'Canada', 'Germany',
       'Great Britain', 'Nigeria', 'Austria', 'Indonesia', 'Belgium',
       'Turkey', 'Slovenia', 'Netherlands', 'Australia', 'France',
       'Poland', "People's Republic of China", 'Denmark', 'ROC',
       'Kazakhstan', 'Portugal', 'Unified Team', 'Estonia', 'Jamaica',
       'New Zealand', 'Spain', 'Libya', 'Morocco', 'Egypt', 'Chile',
       'India', 'Islamic Republic of Iran', 'South Africa', 'Tunisia',
       'Saudi Arabia', 'Cuba', 'Peru', 'Hungary', 'Saar', 'Bulgaria',
       'Romania', 'Luxembourg', 'Bahrain', 'Kenya', 'Qatar', 'Bahamas',
       'Haiti', 'Belarus', 'Ukraine', 'Tajikistan

In [10]:
#Merge des deux datasets
df_hosts.rename({'slug_game':'game_slug'}, axis=1, inplace=True)
df_result.rename({'slug_game':'game_slug'}, axis=1, inplace=True)
df_full = pd.merge(df_result, df_hosts, on='game_slug')


In [11]:
# Nettoyage des données
df_full = df_full.dropna(subset=['country_name'])
df_full

,_c0,Unnamed: 0,discipline_title,event_title,game_slug,participant_type,medal_type,rank_equal,rank_position,country_name,...,value_unit,value_type,index,game_end_date,game_start_date,game_location,game_name,game_season,game_year,start_year
0,55739,55739,Skeleton,Individual men,salt-lake-city-2002,Athlete,None,None,20,Republic of Korea,...,104510,TIME,10,2002-02-24 08:00:00+00:00,2002-02-08 15:00:00+00:00,United States,Salt Lake City 2002,Winter,2002,2002
1,110138,110138,Speed skating,5000m men,innsbruck-1976,Athlete,None,None,11,Republic of Korea,...,464210,TIME,24,1976-02-15 19:00:00+00:00,1976-02-04 07:00:00+00:00,Austria,Innsbruck 1976,Winter,1976,1976
2,55740,55740,Skeleton,Individual men,salt-lake-city-2002,Athlete,None,None,21,Latvia,...,104670,TIME,10,2002-02-24 08:00:00+00:00,2002-02-08 15:00:00+00:00,United States,Salt Lake City 2002,Winter,2002,2002
3,110139,110139,Speed skating,5000m men,innsbruck-1976,Athlete,None,None,12,United States of America,...,468080,TIME,24,1976-02-15 19:00:00+00:00,1976-02-04 07:00:00+00:00,Austria,Innsbruck 1976,Winter,1976,1976
4,55741,55741,Skeleton,Individual men,salt-lake-city-2002,Athlete,None,None,22,Russian Federation,...,105590,TIME,10,2002-02-24 08:00:00+00:00,2002-02-08 15:00:00+00:00,United States,Salt Lake City 2002,Winter,2002,2002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162799,27920,27920,Athletics,1500m men,london-2012,Athlete,BRONZE,None,3,Morocco,...,215130,TIME,5,2012-08-12 19:00:00+00:00,2012-07-27 07:00:00+00:00,Great Britain,London 2012,Summer,2012,2012
162800,27921,27921,Athletics,1500m men,london-2012,Athlete,None,None,4,United States of America,...,215170,TIME,5,2012-08-12 19:00:00+00:00,2012-07-27 07:00:00+00:00,Great Britain,London 2012,Summer,2012,2012
162801,27922,27922,Athletics,1500m men,london-2012,Athlete,None,None,5,Norway,...,215430,TIME,5,2012-08-12 19:00:00+00:00,2012-07-27 07:00:00+00:00,Great Britain,London 2012,Summer,2012,2012
162802,27923,27923,Athletics,1500m men,london-2012,Athlete,None,None,6,Ethiopia,...,215440,TIME,5,2012-08-12 19:00:00+00:00,2012-07-27 07:00:00+00:00,Great Britain,London 2012,Summer,2012,2012


In [12]:
summer_games_data = df_full[df_full['game_season'] == 'Summer']
summer_games_data

,_c0,Unnamed: 0,discipline_title,event_title,game_slug,participant_type,medal_type,rank_equal,rank_position,country_name,...,value_unit,value_type,index,game_end_date,game_start_date,game_location,game_name,game_season,game_year,start_year
15,83310,83310,Table Tennis,Singles men,barcelona-1992,Athlete,None,True,17,Czechoslovakia,...,None,None,15,1992-08-09 18:00:00+00:00,1992-07-25 06:00:00+00:00,Spain,Barcelona 1992,Summer,1992,1992
20,83311,83311,Table Tennis,Singles men,barcelona-1992,Athlete,None,True,17,Independent Olympic Athletes,...,None,None,15,1992-08-09 18:00:00+00:00,1992-07-25 06:00:00+00:00,Spain,Barcelona 1992,Summer,1992,1992
29,83312,83312,Table Tennis,Singles men,barcelona-1992,Athlete,None,True,33,Nigeria,...,None,None,15,1992-08-09 18:00:00+00:00,1992-07-25 06:00:00+00:00,Spain,Barcelona 1992,Summer,1992,1992
36,83313,83313,Table Tennis,Singles men,barcelona-1992,Athlete,None,True,33,Indonesia,...,None,None,15,1992-08-09 18:00:00+00:00,1992-07-25 06:00:00+00:00,Spain,Barcelona 1992,Summer,1992,1992
38,27925,27925,Athletics,1500m men,london-2012,Athlete,None,None,8,Turkey,...,216720,TIME,5,2012-08-12 19:00:00+00:00,2012-07-27 07:00:00+00:00,Great Britain,London 2012,Summer,2012,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162799,27920,27920,Athletics,1500m men,london-2012,Athlete,BRONZE,None,3,Morocco,...,215130,TIME,5,2012-08-12 19:00:00+00:00,2012-07-27 07:00:00+00:00,Great Britain,London 2012,Summer,2012,2012
162800,27921,27921,Athletics,1500m men,london-2012,Athlete,None,None,4,United States of America,...,215170,TIME,5,2012-08-12 19:00:00+00:00,2012-07-27 07:00:00+00:00,Great Britain,London 2012,Summer,2012,2012
162801,27922,27922,Athletics,1500m men,london-2012,Athlete,None,None,5,Norway,...,215430,TIME,5,2012-08-12 19:00:00+00:00,2012-07-27 07:00:00+00:00,Great Britain,London 2012,Summer,2012,2012
162802,27923,27923,Athletics,1500m men,london-2012,Athlete,None,None,6,Ethiopia,...,215440,TIME,5,2012-08-12 19:00:00+00:00,2012-07-27 07:00:00+00:00,Great Britain,London 2012,Summer,2012,2012


In [13]:
# show duplicates
summer_games_data.duplicated().sum()

# drop duplicates
summer_games_data.drop_duplicates()

,_c0,Unnamed: 0,discipline_title,event_title,game_slug,participant_type,medal_type,rank_equal,rank_position,country_name,...,value_unit,value_type,index,game_end_date,game_start_date,game_location,game_name,game_season,game_year,start_year
15,83310,83310,Table Tennis,Singles men,barcelona-1992,Athlete,None,True,17,Czechoslovakia,...,None,None,15,1992-08-09 18:00:00+00:00,1992-07-25 06:00:00+00:00,Spain,Barcelona 1992,Summer,1992,1992
20,83311,83311,Table Tennis,Singles men,barcelona-1992,Athlete,None,True,17,Independent Olympic Athletes,...,None,None,15,1992-08-09 18:00:00+00:00,1992-07-25 06:00:00+00:00,Spain,Barcelona 1992,Summer,1992,1992
29,83312,83312,Table Tennis,Singles men,barcelona-1992,Athlete,None,True,33,Nigeria,...,None,None,15,1992-08-09 18:00:00+00:00,1992-07-25 06:00:00+00:00,Spain,Barcelona 1992,Summer,1992,1992
36,83313,83313,Table Tennis,Singles men,barcelona-1992,Athlete,None,True,33,Indonesia,...,None,None,15,1992-08-09 18:00:00+00:00,1992-07-25 06:00:00+00:00,Spain,Barcelona 1992,Summer,1992,1992
38,27925,27925,Athletics,1500m men,london-2012,Athlete,None,None,8,Turkey,...,216720,TIME,5,2012-08-12 19:00:00+00:00,2012-07-27 07:00:00+00:00,Great Britain,London 2012,Summer,2012,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162799,27920,27920,Athletics,1500m men,london-2012,Athlete,BRONZE,None,3,Morocco,...,215130,TIME,5,2012-08-12 19:00:00+00:00,2012-07-27 07:00:00+00:00,Great Britain,London 2012,Summer,2012,2012
162800,27921,27921,Athletics,1500m men,london-2012,Athlete,None,None,4,United States of America,...,215170,TIME,5,2012-08-12 19:00:00+00:00,2012-07-27 07:00:00+00:00,Great Britain,London 2012,Summer,2012,2012
162801,27922,27922,Athletics,1500m men,london-2012,Athlete,None,None,5,Norway,...,215430,TIME,5,2012-08-12 19:00:00+00:00,2012-07-27 07:00:00+00:00,Great Britain,London 2012,Summer,2012,2012
162802,27923,27923,Athletics,1500m men,london-2012,Athlete,None,None,6,Ethiopia,...,215440,TIME,5,2012-08-12 19:00:00+00:00,2012-07-27 07:00:00+00:00,Great Britain,London 2012,Summer,2012,2012


In [14]:
summer_games_data.medal_type.unique()
summer_games_data.isnull().sum()

_c0                           0
Unnamed: 0                    0
discipline_title              0
event_title                   0
game_slug                     0
participant_type              0
medal_type               101672
rank_equal                90721
rank_position              3858
country_name                  0
country_code               3555
country_3_letter_code         0
athlete_url               26314
athlete_full_name         16658
value_unit                68966
value_type                61920
index                         0
game_end_date                 0
game_start_date               0
game_location                 0
game_name                     0
game_season                   0
game_year                     0
start_year                    0
dtype: int64

In [15]:
# Remove useless columns
summer_games_data.drop(['Unnamed: 0', 'athlete_url', '_c0', 'country_3_letter_code', 'country_code', 'index', 'game_end_date', 'game_start_date', 'start_year', 'game_season', 'game_name'], axis=1, inplace=True)

C:\Users\classico\AppData\Local\Temp\ipykernel_14924\1119707568.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summer_games_data.drop(['Unnamed: 0', 'athlete_url', '_c0', 'country_3_letter_code', 'country_code', 'index', 'game_end_date', 'game_start_date', 'start_year', 'game_season', 'game_name'], axis=1, inplace=True)


In [16]:
summer_games_data.columns

Index(['discipline_title', 'event_title', 'game_slug', 'participant_type',
       'medal_type', 'rank_equal', 'rank_position', 'country_name',
       'athlete_full_name', 'value_unit', 'value_type', 'game_location',
       'game_year'],
      dtype='object')

In [17]:
summer_games_data

,discipline_title,event_title,game_slug,participant_type,medal_type,rank_equal,rank_position,country_name,athlete_full_name,value_unit,value_type,game_location,game_year
15,Table Tennis,Singles men,barcelona-1992,Athlete,None,True,17,Czechoslovakia,Petr KORBEL,None,None,Spain,1992
20,Table Tennis,Singles men,barcelona-1992,Athlete,None,True,17,Independent Olympic Athletes,Zoran KALINIC,None,None,Spain,1992
29,Table Tennis,Singles men,barcelona-1992,Athlete,None,True,33,Nigeria,Atanda Ganiyu MUSA,None,None,Spain,1992
36,Table Tennis,Singles men,barcelona-1992,Athlete,None,True,33,Indonesia,Anton SUSENO,None,None,Spain,1992
38,Athletics,1500m men,london-2012,Athlete,None,None,8,Turkey,Ilham Tanui OZBILEN,216720,TIME,Great Britain,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
162799,Athletics,1500m men,london-2012,Athlete,BRONZE,None,3,Morocco,Abdalaati IGUIDER,215130,TIME,Great Britain,2012
162800,Athletics,1500m men,london-2012,Athlete,None,None,4,United States of America,Matthew CENTROWITZ,215170,TIME,Great Britain,2012
162801,Athletics,1500m men,london-2012,Athlete,None,None,5,Norway,Henrik INGEBRIGTSEN,215430,TIME,Great Britain,2012
162802,Athletics,1500m men,london-2012,Athlete,None,None,6,Ethiopia,Mekonnen GEBREMEDHIN,215440,TIME,Great Britain,2012


In [19]:
summer_games_data.columns

Index(['discipline_title', 'event_title', 'game_slug', 'participant_type',
       'medal_type', 'rank_equal', 'rank_position', 'country_name',
       'athlete_full_name', 'value_unit', 'value_type', 'game_location',
       'game_year'],
      dtype='object')

In [20]:
medals_data = summer_games_data[['country_name', 'medal_type', 'game_slug', 'athlete_full_name']]
medals_data

,country_name,medal_type,game_slug,athlete_full_name
15,Czechoslovakia,None,barcelona-1992,Petr KORBEL
20,Independent Olympic Athletes,None,barcelona-1992,Zoran KALINIC
29,Nigeria,None,barcelona-1992,Atanda Ganiyu MUSA
36,Indonesia,None,barcelona-1992,Anton SUSENO
38,Turkey,None,london-2012,Ilham Tanui OZBILEN
...,...,...,...,...
162799,Morocco,BRONZE,london-2012,Abdalaati IGUIDER
162800,United States of America,None,london-2012,Matthew CENTROWITZ
162801,Norway,None,london-2012,Henrik INGEBRIGTSEN
162802,Ethiopia,None,london-2012,Mekonnen GEBREMEDHIN


In [21]:
# Sélectionner les colonnes pertinentes
medals_data = summer_games_data[['country_name', 'medal_type', 'game_slug', 'athlete_full_name']]

# Extraire l'année des jeux olympiques
medals_data['year'] = medals_data['game_slug'].str.extract(r'(\d{4})').astype(int)

# Dictionnaire correction
country_dict = {
    'German Democratic Republic (Germany)':'Germany',
    'Federal Republic of Germany':'Germany',
    "Democratic People's Republic of Korea":'North Korea',
    "ROC":"Russia",
    "Republic of Korea":"South Korea",
    "People's Republic of China":'China',
    "Islamic Republic of Iran":'Iran',
    "United States of America":'USA'}

medals_data['country_name'] = medals_data['country_name'].replace(country_dict)

# Ajouter une colonne pour le nombre d'athlètes par pays par année
medals_data['num_athletes'] = medals_data.groupby(['country_name', 'year'])['athlete_full_name'].transform('nunique')

# Ajouter une colonne comptant les points de médailles
medals_data['medal_value'] = medals_data['medal_type'].map({'GOLD': 3, 'SILVER': 2, 'BRONZE': 1})

# Ajouter une colonne comptant le nombre de médailles
medals_data['medal_count'] = medals_data['medal_type'].map({'GOLD': 1, 'SILVER': 1, 'BRONZE': 1, 'None': 0})

# Agréger les données par pays et par année
medals_by_country_year = medals_data.groupby(['country_name', 'year', 'num_athletes'])[['medal_value', 'medal_count']].sum().reset_index()

# Renommer les colonnes pour plus de clarté
medals_by_country_year.columns = ['country_name', 'year', 'num_athletes', 'total_medal_value', 'total_medal_count']

C:\Users\classico\AppData\Local\Temp\ipykernel_14924\2350197799.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medals_data['year'] = medals_data['game_slug'].str.extract(r'(\d{4})').astype(int)
C:\Users\classico\AppData\Local\Temp\ipykernel_14924\2350197799.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medals_data['country_name'] = medals_data['country_name'].replace(country_dict)
C:\Users\classico\AppData\Local\Temp\ipykernel_14924\2350197799.py:21: SettingWithCopyWarning: 
A value is trying to

In [22]:
# Séparer les données d'entraînement (jusqu'à 2016) et de test (2020)
# train_data = medals_by_country_year[(medals_by_country_year['year'] >= 1990) & (medals_by_country_year['year'] <= 2016)]
train_data = medals_by_country_year[medals_by_country_year['year'] <= 2016]
test_data = medals_by_country_year[medals_by_country_year['year'] == 2020]

X_train = train_data[['country_name', 'year', 'num_athletes']]
y_train = train_data[['total_medal_value', 'total_medal_count']]
X_test = test_data[['country_name', 'year', 'num_athletes']]
y_test = test_data[['total_medal_value', 'total_medal_count']]

In [23]:
print(medals_by_country_year)

     country_name  year  num_athletes  total_medal_value  total_medal_count
0     Afghanistan  1936             0                0.0                0.0
1     Afghanistan  1948             0                0.0                0.0
2     Afghanistan  1956             0                0.0                0.0
3     Afghanistan  1960            10                0.0                0.0
4     Afghanistan  1964             8                0.0                0.0
...           ...   ...           ...                ...                ...
2879     Zimbabwe  2004             5                6.0                3.0
2880     Zimbabwe  2008            10                9.0                4.0
2881     Zimbabwe  2012             7                0.0                0.0
2882     Zimbabwe  2016            11                0.0                0.0
2883     Zimbabwe  2020             2                0.0                0.0

[2884 rows x 5 columns]


# Linear Regression

In [24]:
# Préparer le transformateur pour l'encodage et la normalisation
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['year', 'num_athletes']),
    ])

# Créer le pipeline avec le préprocesseur et le modèle de régression linéaire
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    # On prédit à la fois le nombre de points de médailles et le nombre médaille avec la régression linéaire multi-sorties
    ('regressor', MultiOutputRegressor(LinearRegression()))
])

# Entraîner le modèle
model.fit(X_train, y_train)

# Prédire les médailles pour 2020
y_pred = model.predict(X_test)

# Évaluer le modèle
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

# Afficher les prédictions
predictions = pd.DataFrame({
    'country_name': X_test['country_name'],
    'predicted_points': y_pred[:, 0],
    'predicted_medals': y_pred[:, 1]
})

# Remplacer les valeurs négatives par 0
predictions['predicted_points'] = predictions['predicted_points'].apply(lambda x: max(0, x))
predictions['predicted_medals'] = predictions['predicted_medals'].apply(lambda x: max(0, x))

# Classer la dataframe dans l'ordre des médailles gagnées par pays
predictions = predictions.sort_values(by='predicted_points', ascending=False)

print("total medals : " + str(predictions['predicted_medals'].sum()))
print(predictions)

Mean Absolute Error: 7.084415474318739
total medals : 1130.9614874502374
     country_name  predicted_points  predicted_medals
545         China        228.060493        113.034897
2711          USA        176.085224         87.313937
1393        Japan        122.253696         60.674372
960       Germany         98.741074         49.038700
911        France         98.741074         49.038700
...           ...               ...               ...
1404       Jordan          0.000000          0.000000
1432     Kiribati          0.000000          0.000000
1434       Kosovo          0.000000          0.000000
1447       Kuwait          0.000000          0.000000
2883     Zimbabwe          0.000000          0.000000

[189 rows x 3 columns]


# Polynomial Regression

In [25]:
# Préparer le transformateur pour l'encodage et la normalisation
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['year', 'num_athletes']),
    ])

polynomial_features = PolynomialFeatures(degree=2, include_bias=False)
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('polynomial_features', polynomial_features),
    ('regressor', MultiOutputRegressor(LinearRegression()))
])

# Entraîner le modèle
model.fit(X_train, y_train)

# Prédire les médailles pour 2020
y_pred = model.predict(X_test)

# Évaluer le modèle
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

# Afficher les prédictions
predictions = pd.DataFrame({
    'country_name': X_test['country_name'],
    'predicted_points': y_pred[:, 0],
    'predicted_medals': y_pred[:, 1]
})

# Remplacer les valeurs négatives par 0
predictions['predicted_points'] = predictions['predicted_points'].apply(lambda x: max(0, x))
predictions['predicted_medals'] = predictions['predicted_medals'].apply(lambda x: max(0, x))

# Classer la dataframe dans l'ordre des médailles gagnées par pays
predictions = predictions.sort_values(by='predicted_points', ascending=False)

print("total medals : " + str(predictions['predicted_medals'].sum()))
print(predictions)

Mean Absolute Error: 4.430006440960208
total medals : 1029.0401325295625
       country_name  predicted_points  predicted_medals
545           China        331.889925        161.339985
2711            USA        221.201336        108.195374
1393          Japan        128.398647         63.385359
911          France         94.837020         47.080288
960         Germany         94.837020         47.080288
...             ...               ...               ...
1085  Guinea-Bissau          0.000000          0.000000
1078         Guinea          0.000000          0.000000
1051           Guam          0.000000          0.000000
1042        Grenada          0.000000          0.000000
2883       Zimbabwe          0.000000          0.000000

[189 rows x 3 columns]


# Random Forest

In [26]:
# Préparer le transformateur pour l'encodage et la normalisation
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['year', 'num_athletes']),
    ])

polynomial_features = PolynomialFeatures(degree=2, include_bias=False)
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('polynomial_features', polynomial_features),
    ('regressor', MultiOutputRegressor(RandomForestRegressor(random_state=42)))
])

# Entraîner le modèle
model.fit(X_train, y_train)

# Prédire les médailles pour 2020
y_pred = model.predict(X_test)

# Évaluer le modèle
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

# Afficher les prédictions
predictions = pd.DataFrame({
    'country_name': X_test['country_name'],
    'predicted_points': y_pred[:, 0],
    'predicted_medals': y_pred[:, 1]
})

# Remplacer les valeurs négatives par 0
predictions['predicted_points'] = predictions['predicted_points'].apply(lambda x: max(0, x))
predictions['predicted_medals'] = predictions['predicted_medals'].apply(lambda x: max(0, x))

# Classer la dataframe dans l'ordre des médailles gagnées par pays
print("total medals : " + str(predictions['predicted_medals'].sum()))
print(predictions.sort_values(by='predicted_medals', ascending=False))

Mean Absolute Error: 3.1150097488890593
total medals : 957.1043716590757
       country_name  predicted_points  predicted_medals
545           China            243.61            119.29
2711            USA            234.02            112.50
1393          Japan            140.25             67.61
151       Australia            105.91             45.68
911          France             87.09             40.41
...             ...               ...               ...
1131       Honduras              0.37              0.00
1119          Haiti              0.37              0.00
1103         Guyana              0.37              0.00
1085  Guinea-Bissau              0.37              0.00
2883       Zimbabwe              0.37              0.00

[189 rows x 3 columns]
